In [41]:
import sys
sys.path.insert(0,"/home2/tathagato/summarization/MACSUM/naacl")
import os
from eval import evaluate
import pickle as pkl
import numpy as np

In [42]:
# all_result_paths = [
#     "/scratch/tathagato/openai_outputs/all_results.pkl",
#     "/scratch/tathagato/openai_outputs/all_results_rest.pkl",
#     "/scratch/tathagato/openai_outputs/all_results_llama.pkl"
# ]
# # read the two pickle files and merge them and save as final_results.pkl
# final_results = {}
# for result_path in all_result_paths:
#     with open(result_path, "rb") as f:
#         results = pkl.load(f)
#         final_results.update(results)
# with open("/scratch/tathagato/openai_outputs/final_results.pkl", "wb") as f:
#     pkl.dump(final_results, f)
# print(len(final_results))

In [67]:
attributes = ['topic']
arguments = [
    ["/scratch/tathagato/naacl/zero_shot/llama_topic.pkl", "llama"],
    ["/scratch/tathagato/naacl/zero_shot/mistral_topic.pkl", "mistral"],
    ["/scratch/tathagato/naacl/single_attribute_sft/llama_topic/results.pkl", "llama"],
    ["/scratch/tathagato/naacl/single_attribute_sft/mistral_topic/results.pkl", "mistral"],
    ["/scratch/tathagato/naacl/single_attribute_dpo/llama_topic/results.pkl", "llama"],
    ["/scratch/tathagato/naacl/single_attribute_dpo/mistral_topic/results.pkl", "mistral"]


]
def generate_latex_table_row(model, method, config, rouge_scores, mean, std):
    """
    Generate a LaTeX table row for the given model information and scores.
    
    :param model: str, name of the model
    :param method: str, method used (e.g., "Finetuned")
    :param config: str, configuration details
    :param rouge_scores: tuple, (rouge_1, rouge_2, rouge_L) scores
    :param mean: float, mean score from GPT-4 evaluation
    :param std: float, standard deviation from GPT-4 evaluation
    :return: str, formatted LaTeX table row
    """
    # Format ROUGE scores
    rouge_str = f"{rouge_scores[0]:.2f} / {rouge_scores[1]:.2f} / {rouge_scores[2]:.2f}"
    
    # Format mean and std
    mean_std_str = f"{mean:.2f} & {std:.2f}"
    
    # Construct the LaTeX table row
    row = f"{model} & {method} & {config} & {rouge_str} & {mean_std_str} \\\\"
    
    return row

def extract_floats(string):
    # This pattern matches numbers with optional negative sign,
    # decimal point, and exponent notation
    pattern = r'-?\d+(?:\.\d+)?(?:[eE][-+]?\d+)?'
    
    # Find all matches in the string
    matches = re.findall(pattern, string)
    
    # Convert matches to float
    floats = [float(match) for match in matches]
    
    return floats
def extract_words_around_then(filepath):
    # Check if "then" is in the filepath
    if "then" not in filepath:
        return ""
    
    # Regular expression to match the pattern
    pattern = r'_([^_]+)_then_([^_]+)_'
    
    # Search for the pattern in the filepath
    match = re.search(pattern, filepath)
    
    if match:
        # Extract the words before and after "then"
        word_before = match.group(1)
        word_after = match.group(2)
        return f"{word_before[0]}->{word_after[0]}"

def extract_meta_information(path):
    config = ""
    model = None
    if "mistral" in path:
        model = "Mistral-7b-v3"
    elif "llama" in path:
        model = "Llama-7b-v1"
    method = "not sure"
    if "zero_shot" in path:
        method = "zero shot"

    if "adapter_fusion" in path:
        method = "adapter fusion"
        w1, w2 = extract_floats(path)[1], extract_floats(path)[2]
        config = f"{w1}, {w2}"

    elif "joint_mult_attribute" in path:
        method = "joint training"
    elif "multi_attribute_single_adapter_continued" in path:
        method = "single adapter continous"
        config = extract_words_around_then(path)
    elif "multi_attribute_multi_adapter" in path:
        method = "multi adapter"
        config = extract_words_around_then(path)
    
    objective = ""
    if "sft" in path:
        objective = "SFT"
    elif "dpo" in path:
        objective = "DPO"
    else:
        objective = "Zero Shot"
    print(model)
    return model, method, objective, config

def get_rouge_score_from_res(res):
    rouge1 = res['topic']['overall']['rouge1']
    rouge2 = res['topic']['overall']['rouge2']
    rougeL = res['topic']['overall']['rougeL']
    return rouge1, rouge2, rougeL

def get_gpt4_evaluation(file):
    gpt_file = get_gpt_file(file)
    gpt_res = pkl.load(open(gpt_file, "rb"))
    scores = np.array([int(res['responses'][0]) for res in gpt_res.values()])
    mean = np.mean(scores)
    std = np.std(scores)
    return mean, std



In [68]:
def get_gpt_file(file):
    gpt_directory = "/scratch/tathagato/openai_outputs/"
    changed_file_name = file.replace("/", "-")[1:]
    gpt_file_path = os.path.join(gpt_directory, changed_file_name)
    #check if the file exists
    if os.path.exists(gpt_file_path):
        return gpt_file_path
    else:
        print(f"File {gpt_file_path} does not exist")
        return None
rows = []
for file , mod in arguments:
    path = file[0]
    res = evaluate(file, model = mod, attributes=attributes)
    rouge_1, rouge_2, rouge_L = get_rouge_score_from_res(res)
    mean, std = get_gpt4_evaluation(file)
    model, method, objective, config = extract_meta_information(file)
    print(model)
    rouge_scores = (rouge_1, rouge_2, rouge_L)
    row = generate_latex_table_row(model, objective, config, rouge_scores, mean, std)
    rows.append(row)

    


evaluating llama with attributes ['topic']
length of data is  266
/scratch/tathagato/naacl/zero_shot/llama_topic.pkl is being evaluated on topic


/home2/tathagato/miniconda3/envs/roy/lib/python3.9/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


--------------overall----------------
rouge1 : 0.297424114374473
rouge2 : 0.09756386448695305
rouge3 : 0.041503094260232735
rougeL : 0.19780125375875435
number : 266
-------------------------------------------------
Llama-7b-v1
Llama-7b-v1
evaluating mistral with attributes ['topic']
length of data is  266
/scratch/tathagato/naacl/zero_shot/mistral_topic.pkl is being evaluated on topic
--------------overall----------------
rouge1 : 0.16429442494439167
rouge2 : 0.058629450545901546
rouge3 : 0.027004938168936524
rougeL : 0.11200006088617187
number : 266
-------------------------------------------------
Mistral-7b-v3
Mistral-7b-v3
evaluating llama with attributes ['topic']
length of data is  266
/scratch/tathagato/naacl/single_attribute_sft/llama_topic/results.pkl is being evaluated on topic
--------------overall----------------
rouge1 : 0.283551122023257
rouge2 : 0.09422355412377852
rouge3 : 0.04247932438602692
rougeL : 0.19088838990584622
number : 266
-----------------------------------

In [69]:
for row in rows:
    print(row)

Llama-7b-v1 & Zero Shot &  & 0.30 / 0.10 / 0.20 & 4.30 & 0.58 \\
Mistral-7b-v3 & Zero Shot &  & 0.16 / 0.06 / 0.11 & 4.45 & 0.64 \\
Llama-7b-v1 & SFT &  & 0.28 / 0.09 / 0.19 & 4.21 & 0.63 \\
Mistral-7b-v3 & SFT &  & 0.22 / 0.06 / 0.15 & 3.79 & 0.91 \\
Llama-7b-v1 & DPO &  & 0.31 / 0.10 / 0.20 & 4.23 & 0.65 \\
Mistral-7b-v3 & DPO &  & 0.22 / 0.08 / 0.15 & 4.47 & 0.63 \\


In [6]:
final_res = {}
for file, model in arguments:
    print(file, model)
    try:
        res = evaluate(file, model=model, attributes=attributes)
        print(res)
        final_res[file] = {"result": res, "model": model}
    except Exception as e:
        print(f"Error evaluating {file}: {e}")
        res = None
    


/scratch/tathagato/naacl/zero_shot/llama_topic.pkl llama
evaluating llama with attributes ['topic']
length of data is  266
/scratch/tathagato/naacl/zero_shot/llama_topic.pkl is being evaluated on topic


/home2/tathagato/miniconda3/envs/roy/lib/python3.9/site-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(io.BytesIO(b))


--------------overall----------------
rouge1 : 0.297424114374473
rouge2 : 0.09756386448695305
rouge3 : 0.041503094260232735
rougeL : 0.19780125375875435
number : 266
-------------------------------------------------
{'topic': {'rouge_raw': {'aggregrated_score': {'rouge1': {'precision': 0.48910092319319026, 'recall': 0.23706548347418893, 'f1': 0.297424114374473}, 'rouge2': {'precision': 0.16044505409211648, 'recall': 0.07796161259238481, 'f1': 0.09756386448695302}, 'rouge3': {'precision': 0.0691704841765913, 'recall': 0.033189876613137964, 'f1': 0.041503094260232756}, 'rougeL': {'precision': 0.3300728042526286, 'recall': 0.1569563376970118, 'f1': 0.19780125375875432}}, 'results': [{'rouge1': {'precision': 0.4915254237288136, 'recall': 0.48333333333333334, 'f1': 0.48739495798319327}, 'rouge2': {'precision': 0.1724137931034483, 'recall': 0.1694915254237288, 'f1': 0.17094017094017094}, 'rouge3': {'precision': 0.03508771929824561, 'recall': 0.034482758620689655, 'f1': 0.034782608695652174},

In [13]:
for key in final_res:
    print(final_res[key]['result']['topic']['overall']['rouge1'])
    break

0.297424114374473


In [7]:
for file in final_res:
    print(file)
    model = final_res[file]["model"]
    print(model)
    display_length(final_res[file]["result"])
    print("")
    print("----------------------------------------------")

/scratch/tathagato/naacl/zero_shot/llama_topic.pkl
llama


NameError: name 'display_length' is not defined